# THIS NOTEBOOK IS DEDICATED TO THE ANALYSIS OF 

# *SEQ SCORE / WAITING TIME*


In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
from IPython.display import clear_output, display, HTML, Image
import matplotlib.cm as cm
import warnings
import types
warnings.filterwarnings("ignore")
import sys, time
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib import mlab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from pylab import *
import matplotlib.backends.backend_pdf
from sklearn.decomposition import KernelPCA

ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
CWD=os.getcwd()
os.chdir(CommonNoteBookesPath)
root='/data'
%run UtilityTools.ipynb
%run Animal_Tags.ipynb
%run Lesion_Size.ipynb
%run BatchRatBehavior.ipynb
%run plotRat_documentation_3_KinematicsInvestigation.ipynb
%run plotRat_documentation_1_GeneralBehavior.ipynb
%run loadRat_documentation.ipynb
%run RunBatchRat_3_CompareGroups.ipynb
os.chdir(CWD)

param={
    "binSize":0.25,
    "trialOffset":20., #max end of trial, in seconds (position will be cutted)
    "sigmaSmoothPosition":0.1,  #smooth the position
    #"sigmaSmoothPosition":0.33 for pavel dataType
    "sigmaSmoothSpeed":0.3, #smooth the speed
    "positionDiffRange": [2.,5.], #min and max differences allowed between two consecutive positions
                                  #min to correct start, max to correct jumps
    "pawFrequencyRange":[2.,10.],
    "startAnalysisParams":[10,0.2,0.5],
    "cameraToTreadmillDelay":2., #seconds, usual time between camera start and treadmill start
    "nbJumpMax" : 100., #if jumps>nbJumpMax, trial is badly tracked
    
    
    #parameter to detect end of trial (first position minima)
    "endTrial_backPos":55,  # minima is after the animal went once to the back (after first time position>backPos)
    "endTrial_frontPos":30, # minima's position is in front of treadmill (position[end]<frontPos)
    "endTrial_minTimeSec":4, # minima is after minTimeSec seconds (time[end]>minTimeSec)
    }
Y1,Y2=(0,90)
print('Done')

## motor seq score

In [ ]:
def plot_event_1on1(root, SessionDict, TaskParamToPlot,
                    ax=None, preSlice=slice(-5,None), postSlice=slice(0,5)):
    
    if ax is None:
        ax=plt.gca()
    
    
    Results,nSessionPre,nSessionPost=event_statistic(root,
                                                     SessionDict,
                                                     parameter=param,
                                                     redo=False,
                                                     TaskParamToPlot=[TaskParamToPlot])

    data=np.array(list(Results[TaskParamToPlot].values()))
    slicedPreData  = np.ones((data.shape))*np.nan
    slicedPostData = np.ones((data.shape))*np.nan
    
    for row,Dnan in enumerate(data):
        Dpre= Dnan[:nSessionPre][~ np.isnan(Dnan[:nSessionPre])]
        Dpost=Dnan[nSessionPre:][~ np.isnan(Dnan[nSessionPre:])]
        preData =Dpre[preSlice]
        postData=Dpost[postSlice]

        slicedPreData[row,:len(preData)]=preData
        slicedPostData[row,:len(postData)]=postData
    
    y0=np.nanmean(slicedPreData,axis=1)
    y1=np.nanmean(slicedPostData,axis=1)
    
    y=np.array([y0,y1]).T
    
    #plotting the errorbar
    props={'color':'k','linewidth':1}
    ax.boxplot(x=y,whis=[5,95],
                  positions=[-.5,.5], widths=.3,
                  showcaps=False, showfliers=False,
                  medianprops=props, boxprops=props, whiskerprops=props,
                  zorder=2)

    #plotting single animals
    for before,after in y:
        ax.plot([-.5,.5],[before,after],lw=.5,alpha=1,zorder=1)

    return y, slicedPreData, slicedPostData

In [ ]:
profile1={'Type':'Good',
         'rewardType':'Progressive',
         'initialSpeed':['0','10'],
         'Speed':'10',
         'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak']
         }
profile2={'Type':'Good',
         'rewardType':'Progressive',
         'initialSpeed':['0','10'],
         'Speed':'10',
         'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS','Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
         }

Profiles=(profile1,profile2)
TaskParamToPlot=["Motor Sequence Score","Maximum Position"]
preSlice=slice(-2,None)
postSlice=slice(0,3)

In [ ]:
F=plt.figure(1)
ax=F.add_subplot(111)

_,SessionDict=event_detect(root, Profiles[0], Profiles[1])
y, nSessionPre, slicedPostData=plot_event_1on1(root,SessionDict, TaskParamToPlot=TaskParamToPlot[0],ax=ax,
                preSlice=preSlice, postSlice=postSlice);
clear_output()

In [ ]:
for i,animal in enumerate(SessionDict.keys()):
    print(i, animal,y[i,0],y[i,1],y[i,1]-y[i,0] ,sep='\t')
#     if y[i,1] < 0.4:
#         print(i, animal, y[i,0],y[i,1], sep='\t')


In [ ]:
badAnimalList=[]
for i,animal in enumerate(SessionDict.keys()):
    if y[i,0] < 0.5 and y[i,1] < 0.5: #low seq score
        badAnimalList.append(animal)

F=plt.figure(2)
ax=F.add_subplot(111)

_,SessionDict1=event_detect(root, Profiles[0], Profiles[1],badAnimals=badAnimalList)
y1, nSessionPre1, slicedPostData1=plot_event_1on1(root,SessionDict1, TaskParamToPlot=TaskParamToPlot[1],ax=ax,
                                                  preSlice=slice(-2,None), postSlice=slice(0,3));
clear_output()
F

In [ ]:
for i,animal in enumerate(SessionDict1.keys()):
    print(i, animal,y1[i,0],y1[i,1],y1[i,1]-y1[i,0] ,sep='\t')
#     if y[i,1] < 0.4:
#         print(i, animal, y[i,0],y[i,1], sep='\t')


In [ ]:
x=y1[:,0]-y1[:,1]
# x[y[:,0]<3.5]=np.nan
x=x[~ np.isnan(x)]
q=np.random.normal(loc=0.0, scale=np.std(x), size=(500,1))
p=permtest(x=x, y=q, iterN=10000)

In [ ]:
p.p_val

In [ ]:
plt.hist(x,50,density=True);

In [ ]:
session='Rat223_2018_05_28_10_45'
data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
w,n=waiting_time(data);
print(np.mean(w)*(len(n)/data.nTrial))

In [ ]:
session ='Rat224_2018_04_10_11_07'
data    =Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
seqTrials,fireTimes=sequential_trials(data)